**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install scikit-video
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> ```epsilon``` is important as it allows the agent to act randomly - every $1/\epsilon$ action. It is the simplest way to enforce exploration in the agent behaviour.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 ## /!\ Correct wrong index here

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 ## /!\ Correct wrong index here
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> - ```self.position```: a $N\times N$ matrix representing the cells accessible to the rat. -1 for inaccessible cells, 1 for the cell the rat is located on and 0 for all other cells.
> -  ```self.board```: a $N\times N$ matrix representing the remaining reward on the board along the rat's exploration of the grid.  +0.5 in cells containing cheese, -1 in cells containing poison, 0 in remaining cells.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        s = env.reset()
        game_over = False
        win = 0
        lose = 0
        
        # Execute one game, acumulate reward
        while not game_over:
          a = agent.learned_act(s)
          s, r, game_over = env.act(a)
          
          if r > 0:
            win += r
          elif r < 0:
            lose -= r

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 6.0/12.0. Average score (-6.0)
Win/lose count 8.0/18.0. Average score (-8.0)
Win/lose count 5.0/6.0. Average score (-5.666666666666667)
Win/lose count 6.5/11.0. Average score (-5.375)
Win/lose count 6.5/19.0. Average score (-6.8)
Win/lose count 10.0/12.0. Average score (-6.0)
Win/lose count 11.0/16.0. Average score (-5.857142857142857)
Win/lose count 9.5/12.0. Average score (-5.4375)
Win/lose count 11.0/10.0. Average score (-4.722222222222222)
Win/lose count 9.0/16.0. Average score (-4.95)
Win/lose count 7.0/15.0. Average score (-5.2272727272727275)
Final score: -5.2272727272727275


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




Using the difinition of Q (stated first), and expanding one step the espectancy, one can show the first formula:
\begin{align*}
Q^\pi (s, a) &= E_\pi \left[ \sum_{k=0}^\infty \gamma^k r_{t+k+1} | s_t = s, a_t = a \right] \\
&= E_\pi \left[ r(s,a) + \sum_{k=1}^\infty \gamma^k r_{t+k+1}| s_t = s, a_t = a \right] \\
&= E_{(s', a') \sim p(\cdot |s,a)} \left[ E_{\pi} \left[r(s, a) + \sum_{k=1}^\infty \gamma^k r_{t+k+1} |s_t = s, a_t = a , s_{t+1}=s', a_{t+1}=a' \right] | s_t = s, a_t = a \right] \\
&= E_{(s', a') \sim p(\cdot |s,a)} \left[ r(s, a) + E_{\pi} \left[ \sum_{k=1}^\infty \gamma^k r_{t+k+1} |s_{t+1}=s', a_{t+1}=a' \right] | s_t = s, a_t = a \right] \\
&= E_{(s', a') \sim p(\cdot |s,a)} \left[ r(s, a) + \gamma E_{\pi} \left[ \sum_{k=0}^\infty \gamma^k r_{t+k+1} |s_t=s', a_t=a' \right]  \right] \\
&= E_{(s', a') \sim p(\cdot |s,a)} \left[ r(s, a) + \gamma Q^\pi(s', a') \right] \\
&= E_{(s', a') \sim p(\cdot |s,a)} \left[ r(s, a) + \gamma Q^\pi(s', a') \right] \\
\end{align*}

---

Considering the optimal policy, by definition at each time step $t$, one has: $a_{t} \in \arg\max_{a} Q(s_t, a)$. Injecting this in the previous equation, the expectancy acts on the remaining random variable $s_t$, and we ave:
\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

---

The goal is to learn a representative parametric approximation $Q(\cdot, \cdot, \theta)$ to the action-value function. In order to do so, we want to enforce the fixed point equation characterizing the otpimal action value-function stated directly above. Therefore, for all available tuple of state, action, reward and next state $(s, a, r, s')$, we want to penalize the distance between $Q(s, a)$ and $r(s, a) + \gamma \max_a Q(s', a)$. There is no expectancy in the second expression since in practice we don't have access to several trajectories starting from each state without generating the said trajectories.

Hence the final objective as the sum over all tuples $(s, a, r, s')$ of the squared difference (one type of differentiable distance), for the parametric approximator of the $Q$ function:
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
      # if memory is full, remove first element of memory (older)
      if len(self.memory) > self.max_memory:
        # ind = np.random.randint(len(self.memory))
        self.memory.pop(0)
      
      # append at the end of the list the last element
      self.memory.append(m)

    def random_access(self):
      ind = np.random.randint(len(self.memory))
      return self.memory[ind]
      

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return self.model.predict( np.expand_dims(s, axis=0) ).argmax()

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s, n_s, a, r, game_over = self.memory.random_access()
            
            input_states[i] = s
            target_q[i] = self.model.predict( np.expand_dims(s, axis=0) )[0]
            
            if game_over_:
                target_q[i, a] = r
            else:
                target_q[i, a] = r + self.discount * np.max(self.model.predict( np.expand_dims(n_s, axis=0) )[0])

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential([
            Flatten(input_shape=(5, 5, self.n_state,)),
            Dense(30, activation ='relu'),
            Dense(4, activation = 'sigmoid')
        ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
epochs_train = 51
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train'+str(epochs_train - epochs_train%10)+'.mp4'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 000/051 | Loss 0.0782 | Win/lose count 3.5/9.0 (-5.5)
Epoch 001/051 | Loss 0.1089 | Win/lose count 1.5/1.0 (0.5)
Epoch 002/051 | Loss 0.0887 | Win/lose count 7.0/8.0 (-1.0)
Epoch 003/051 | Loss 0.0985 | Win/lose count 5.5/3.0 (2.5)
Epoch 004/051 | Loss 0.1014 | Win/lose count 6.0/9.0 (-3.0)
Epoch 005/051 | Loss 0.0879 | Win/lose count 6.0/5.0 (1.0)
Epoch 006/051 | Loss 0.1051 | Win/lose count 5.5/4.0 (1.5)
Epoch 007/051 | Loss 0.0824 | Win/lose count 4.0/5.0 (-1.0)
Epoch 008/051 | Loss 0.0801 | Win/lose count 4.0/5.0 (-1.0)
Epoch 009/051 | Loss 0.0978 | Win/lose count 4.0/0 (4.0)
Epoch 010/051 | Loss 0.0711 | Win/lose count 2.5/5.0 (-2.5)
Epoch 011/051 | Loss 0.0723 | Win/lose count 7.0/8.0 (-1.0)
Epoch 012/051 | Loss 0.0929 | Win/lose count 7.0/7.0 (0.0)
Epoch 013/051 | Loss 0.0674 | Win/lose count 6.5/3.0 (3.5)
Epoch 014/051 | Loss 0.0701 | Win/lose count 2.5/

After several tryouts, it appears a simple convolutional network having **one hidden layer** yields the best Q approximator given the training conditions (the memory size remained fixed to 2000 items, batch size of 32 items). 

Tests on the **```epsilon```** parameter show that the value **0.1** works well, with th etraining values given before. More precisely, it is the value that yield the best test performance. Too small an epsilon will cause training to be long as 'exploration' random actions will not happen frequently, so that the Q function will be likely to only represent short term greedy trajectories. On the other hand, too high an epsilon will inject many random actions in the trajectories used for training, so that the proportion of trajectories yielding high return, the ones we want to learn, will be under-represented in the memory and therefore not be learned by reinforce.

Test on the **```learning rate```** parameter show that **0.1** value yield a training relatively rapid, while reaching satisfying objectives.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential([
            Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'),
            Conv2D(30,(2,2),activation='relu'),
            Flatten(),
            Dense(4, activation='softmax')
        ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/051 | Loss 0.0244 | Win/lose count 1.0/1.0 (0.0)
Epoch 001/051 | Loss 0.0246 | Win/lose count 3.5/3.0 (0.5)
Epoch 002/051 | Loss 0.0337 | Win/lose count 4.0/4.0 (0.0)
Epoch 003/051 | Loss 0.0199 | Win/lose count 3.0/1.0 (2.0)
Epoch 004/051 | Loss 0.0193 | Win/lose count 2.5/1.0 (1.5)
Epoch 005/051 | Loss 0.0305 | Win/lose count 5.0/2.0 (3.0)
Epoch 006/051 | Loss 0.0187 | Win/lose count 4.5/3.0 (1.5)
Epoch 007/051 | Loss 0.0205 | Win/lose count 3.0/3.0 (0.0)
Epoch 008/051 | Loss 0.0305 | Win/lose count 0.5/1.0 (-0.5)
Epoch 009/051 | Loss 0.0198 | Win/lose count 4.5/6.0 (-1.5)
Epoch 010/051 | Loss 0.0315 | Win/lose count 2.0/4.0 (-2.0)
Epoch 011/051 | Loss 0.0201 | Win/lose count 5.0/7.0 (-2.0)
Epoch 012/051 | Loss 0.0205 | Win/lose count 3.0/2.0 (1.0)
Epoch 013/051 | Loss 0.0438 | Win/lose count 8.5/4.0 (4.5)
Epoch 014/051 | Loss 0.0197 | Win/lose count 3.0/4.0 (-1.0)
Epoch 015/051 | Loss 0.0205 | Win/lose count 4.0/7.0 (-3.0)
Epoch 016/051 | Loss 0.0201 | Win/lose count 7.0/4

`epsilon` and `learning rate` parameters were set to 0.1 as they gave good results for the fully connected layer. The retained architecture is made of two convolutional layers, respectively comprised of 50 and 25 2x2 filters, which should be largely enough to extract information of a 5x5x2 input grid (4x4x2 = 32 filters necessary to decompose this state). 

Results are superior to the fully connected network, which is not surprising since the rat decisions should not depend on its absolute position but only relative position of reward around it, which convolutional layer allow to encode 'for free'.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.9)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 12.5/0. Average score (12.5)
Win/lose count 10.5/0. Average score (11.5)
Win/lose count 6.5/0. Average score (9.833333333333334)
Win/lose count 5.5/0. Average score (8.75)
Win/lose count 6.0/0. Average score (8.2)
Win/lose count 3.0/0. Average score (7.333333333333333)
Win/lose count 11.0/0. Average score (7.857142857142857)
Win/lose count 7.5/0. Average score (7.8125)
Win/lose count 9.5/0. Average score (8.0)
Win/lose count 11.0/0. Average score (8.3)
Win/lose count 3.0/0. Average score (7.818181818181818)
Final score: 7.818181818181818
Test of the FC
Win/lose count 3.0/0. Average score (3.0)
Win/lose count 6.5/0. Average score (4.75)
Win/lose count 5.0/0. Average score (4.833333333333333)
Win/lose count 2.0/0. Average score (4.125)
Win/lose count 2.5/0. Average score (3.8)
Win/lose count 2.5/0. Average score (3.5833333333333335)
Win/lose count 3.5/0. Average score (3.5714285714285716)
Win/lose count 5.0/0. Average score (3.75)
Win/lose count 3.0/0. Aver

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix='', decay_eps=0.7):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon * decay_eps)


        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.already_visited = np.zeros((grid_size,grid_size)) # EXPLORE: array encodes visited positions
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 ## /!\ Correct wrong index here

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1     
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.already_visited[self.x, self.y]
        self.already_visited[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.already_visited.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 ## /!\ Correct wrong index here
        self.board[self.x,self.y] = 0
        self.t = 0

        self.already_visited = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.already_visited.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [21]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore', decay_eps=0.9)
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/051 | Loss 0.0533 | Win/lose count 5.0/28.900000000000095 (-23.900000000000095)
Epoch 001/051 | Loss 0.0298 | Win/lose count 5.0/20.80000000000004 (-15.80000000000004)
Epoch 002/051 | Loss 0.0403 | Win/lose count 7.5/21.2 (-13.7)
Epoch 003/051 | Loss 0.0398 | Win/lose count 6.0/20.700000000000024 (-14.700000000000024)
Epoch 004/051 | Loss 0.0530 | Win/lose count 9.5/18.3 (-8.8)
Epoch 005/051 | Loss 0.0277 | Win/lose count 7.0/20.90000000000004 (-13.900000000000041)
Epoch 006/051 | Loss 0.0276 | Win/lose count 3.5/18.900000000000002 (-15.400000000000002)
Epoch 007/051 | Loss 0.0410 | Win/lose count 6.0/19.90000000000002 (-13.90000000000002)
Epoch 008/051 | Loss 0.0293 | Win/lose count 4.5/20.100000000000016 (-15.600000000000016)
Epoch 009/051 | Loss 0.0288 | Win/lose count 9.5/17.999999999999993 (-8.499999999999993)
Epoch 010/051 | Loss 0.0287 | Win/lose count 5.5/23.300000000000068 (-17.800000000000068)
Epoch 011/051 | Loss 0.0300 | Win/lose count 3.5/20.40000000000003 (-16.9

In [22]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 3.0/2.0. Average score (1.0)
Win/lose count 3.5/0. Average score (2.25)
Win/lose count 6.5/0. Average score (3.6666666666666665)
Win/lose count 3.0/0. Average score (3.5)
Win/lose count 4.0/3.0. Average score (3.0)
Win/lose count 1.5/0. Average score (2.75)
Win/lose count 4.0/1.0. Average score (2.7857142857142856)
Win/lose count 5.0/1.0. Average score (2.9375)
Win/lose count 7.0/2.0. Average score (3.1666666666666665)
Win/lose count 8.0/1.0. Average score (3.55)
Win/lose count 6.0/1.0. Average score (3.6818181818181817)
Final score: 3.6818181818181817


Using both fully connected and convolutional networks with optimal train parameters derived before, I trained them for several values of the temperature parameter. 

| Temperature | Test score for FC | Test score for CNN |
| :- | :--: | -: | 
| **0.1** | 1.05  | 2.4   |
| **0.3** | 3.7   | 7.8  |
| **0.5** | 7.8   | 15.7 |
| **0.7** | 10.5  | 28.9   |
| **0.9** | 13.8  | 51.3 |

As could be expected, both algorithms yield increasing returns with increasing temperatures. Besides, the CNN model performs uniformly better than the fully connected one. This is not surprising given the two dimensional structure of the state, well-suited to convolutional layers that can learn filters independently of absolute position.

It can be noted that both algorithms perform rather poorly with a small temperature parameter, that is when reward is distributed sparsely on the space. This is likely due to situations where the rat does not see any positive reward, and for which the policy does not integrate a mecanism for exploring the remainder of the board in these situations, since it is already provided by the $\epsilon$-greedy policy used at training. At test time however, a fully greedy policy is used, which explains the poor performance.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***